# Principal Component Analysis (PCA) - Einfach erklärt

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_19/pca_examples.ipynb)

In diesem Notebook lernen wir **PCA (Principal Component Analysis)** kennen – ein mächtiges Verfahren zur **Dimensionsreduktion**.

**Das Problem:** Hochdimensionale Daten (viele Features) sind schwer zu verstehen und zu visualisieren. Menschen können maximal 3D-Daten "sehen".

**Die Lösung:** PCA findet die "wichtigsten Richtungen" in den Daten und reduziert sie auf weniger Dimensionen (2D oder 3D wenn Visualisierung das Ziel ist), ohne zu viel Information zu verlieren.

**Heute lernen wir:**
1. 🎯 PCA mit künstlichen 2D→1D Daten (einfach zu verstehen)
2. 🍷 PCA mit dem Wine-Datensatz 13D→2D (realistisch)
3. 📊 Wie viel Information behalten wir? (Explained Variance)
4. 🔍 k-Means + PCA Kombination (das Dream-Team!)

## 1. Bibliotheken importieren

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, silhouette_score

# Für schönere Plots
plt.style.use('default')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Bibliotheken erfolgreich importiert")
print("🎓 Bereit für PCA-Learning!")

## 2. Teil A: PCA mit künstlichen Daten verstehen

Wir starten mit einem **super einfachen Beispiel**: 2D-Daten, die hauptsächlich in eine Richtung "zeigen". PCA soll diese Hauptrichtung finden.

### Warum künstliche Daten perfekt zum Lernen sind:
- ✅ **Wir kennen die "richtige" Antwort** 
- ✅ **Wir kontrollieren das Problem** (Richtung, Noise-Level)
- ✅ **Wir verstehen, was PCA tut** (keine Black Box)
- ✅ **Visualisierung ist möglich** (2D → 1D)

In [ ]:
# Künstliche Daten erstellen: Punkte entlang einer Diagonale + etwas Rauschen
np.random.seed(42)

# Hauptrichtung: Diagonale (1, 1) - beide Features korreliert
n_points = 200
main_direction = np.array([1, 1])  # 45°-Linie

# Punkte entlang der Hauptrichtung erstellen
t = np.linspace(-3, 3, n_points)
X_clean = np.outer(t, main_direction)  # Perfekte Linie

# Kleines Rauschen hinzufügen (senkrecht zur Hauptrichtung)
noise_direction = np.array([-1, 1])  # Senkrecht zu (1,1)
noise = np.random.normal(0, 0.3, n_points)
X_noisy = X_clean + np.outer(noise, noise_direction) * 0.5

print("📊 Künstliche Daten erstellt:")
print(f"   • {n_points} Datenpunkte")
print(f"   • Hauptrichtung: {main_direction} (45°-Linie)")
print("   • Etwas Rauschen hinzugefügt für Realismus")
print(f"   • Daten-Shape: {X_noisy.shape}")

In [ ]:
# Originaldaten visualisieren
plt.figure(figsize=(12, 4))

# Plot 1: Perfekte Daten (ohne Rauschen)
plt.subplot(1, 3, 1)
plt.scatter(X_clean[:, 0], X_clean[:, 1], alpha=0.6, s=50)
plt.title('Perfekte Daten\n(ohne Rauschen)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)
plt.axis('equal')

# Plot 2: Realistische Daten (mit Rauschen)
plt.subplot(1, 3, 2)
plt.scatter(X_noisy[:, 0], X_noisy[:, 1], alpha=0.6, s=50, c='orange')
plt.title('Realistische Daten\n(mit etwas Rauschen)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)
plt.axis('equal')

# Plot 3: Beide überlagert
plt.subplot(1, 3, 3)
plt.scatter(X_noisy[:, 0], X_noisy[:, 1], alpha=0.6, s=50, label='Mit Rauschen', c='orange')
plt.scatter(X_clean[:, 0], X_clean[:, 1], alpha=0.4, s=30, label='Original (perfekt)', c='blue')
plt.title('Vergleich:\nPerfekt vs. Realistisch')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')

plt.tight_layout()
plt.show()

print("👁️ Mit bloßem Auge sieht man: Die Daten haben eine klare Hauptrichtung!")
print("🤖 PCA soll diese Richtung automatisch finden...")

### PCA anwenden: Die "Hauptrichtung" finden

In [ ]:
# PCA auf die verrauschten Daten anwenden
pca_2d = PCA(n_components=2)  # Alle Komponenten behalten (für Analyse)
X_pca_full = pca_2d.fit_transform(X_noisy)

# PCA nur auf 1D reduzieren (Hauptkomponente)
pca_1d = PCA(n_components=1)
X_pca_1d = pca_1d.fit_transform(X_noisy)

print("🔍 PCA-Analyse Ergebnisse:")
print(f"   • PC1-Richtung: {pca_2d.components_[0]}")
print(f"   • PC2-Richtung: {pca_2d.components_[1]}")
print(f"   • Wahre Hauptrichtung war: {main_direction / np.linalg.norm(main_direction)}")
print()
print("📊 Explained Variance (Erklärte Varianz):")
for i, var in enumerate(pca_2d.explained_variance_ratio_):
    print(f"   • PC{i+1}: {var:.3f} ({var*100:.1f}%)")
print(f"   • Total: {pca_2d.explained_variance_ratio_.sum():.3f} (100%)")
print()
print("💡 PC1 erklärt den größten Teil der Varianz - das ist unsere Hauptrichtung!")

In [ ]:
# Visualisierung: Originaldaten + PCA-Komponenten
plt.figure(figsize=(15, 5))

# Plot 1: Originaldaten mit PCA-Richtungen
plt.subplot(1, 3, 1)
plt.scatter(X_noisy[:, 0], X_noisy[:, 1], alpha=0.6, s=50, c='orange')

# PCA-Zentrum (Mittelwert)
center = np.mean(X_noisy, axis=0)
plt.scatter(center[0], center[1], c='red', s=100, marker='x', linewidth=3, label='Zentrum')

# PCA-Hauptkomponenten als Pfeile
scale = 3 * np.sqrt(pca_2d.explained_variance_)
for i, (component, var, color) in enumerate(zip(pca_2d.components_, scale, ['red', 'green'])):
    plt.arrow(center[0], center[1], 
              component[0] * var, component[1] * var,
              head_width=0.2, head_length=0.3, fc=color, ec=color, linewidth=2,
              label=f'PC{i+1} ({pca_2d.explained_variance_ratio_[i]:.1%})')

plt.title('Originaldaten + PCA-Komponenten')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')

# Plot 2: Daten im PCA-Koordinatensystem
plt.subplot(1, 3, 2)
plt.scatter(X_pca_full[:, 0], X_pca_full[:, 1], alpha=0.6, s=50, c='purple')
plt.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='PC2 = 0')
plt.axvline(x=0, color='red', linestyle='--', alpha=0.5, label='PC1 = 0')
plt.title('Daten in PCA-Koordinaten\n(rotiert)')
plt.xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.1%})')
plt.ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.1%})')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Nur Hauptkomponente (1D-Projektion)
plt.subplot(1, 3, 3)
plt.scatter(X_pca_1d, np.zeros_like(X_pca_1d), alpha=0.6, s=50, c='darkblue')
plt.title('1D-Projektion\n(nur PC1)')
plt.xlabel(f'PC1 ({pca_1d.explained_variance_ratio_[0]:.1%})')
plt.ylabel('0 (andere Dimensionen weg)')
plt.grid(True, alpha=0.3)
plt.ylim(-0.5, 0.5)

plt.tight_layout()
plt.show()

print("🎯 PCA hat die Hauptrichtung gefunden!")
print(f"   PC1 erklärt {pca_1d.explained_variance_ratio_[0]:.1%} der Varianz")
print(f"   Mit nur 1D behalten wir {pca_1d.explained_variance_ratio_[0]:.1%} der Information!")

## 3. Teil B: PCA mit dem Wine-Datensatz (13D → 2D)

Jetzt wird es ernst! Der **Wine-Datensatz aus sklearn** hat **13 chemische Eigenschaften** - viel zu viele für Visualisierung. PCA hilft uns, die wichtigsten 2 Dimensionen zu finden.

### Das Szenario:
- 🍷 **178 Weine** mit **13 chemischen Eigenschaften**
- 🏭 **3 verschiedene Produzenten** (aber PCA weiß das nicht!)
- 🎯 **Ziel:** 13D → 2D reduzieren, aber Gruppierung beibehalten

In [ ]:
# Wine-Datensatz laden
wine = load_wine()
X_wine = wine.data
y_wine = wine.target

print("🍷 Wine-Datensatz Overview:")
print(f"   • Anzahl Weine: {X_wine.shape[0]}")
print(f"   • Anzahl Features: {X_wine.shape[1]}")
print(f"   • Produzenten: {wine.target_names}")
print()

# Echte Verteilung anzeigen
unique, counts = np.unique(y_wine, return_counts=True)
print("🏭 Produzenten-Verteilung:")
for i, (name, count) in enumerate(zip(wine.target_names, counts)):
    print(f"   • {name}: {count} Weine")

print()
print("🔬 Erste 5 chemische Eigenschaften:")
for i, feature in enumerate(wine.feature_names[:5]):
    print(f"   {i+1}. {feature}")
print("   ... und 8 weitere Features")

In [ ]:
# Skalierungsproblem zeigen (wie im k-Means Beispiel)
print("⚠️ Skalierungsproblem demonstrieren:")
print("Feature-Bereiche (min - max):")
for i in range(min(5, X_wine.shape[1])):
    feature_name = wine.feature_names[i]
    min_val = X_wine[:, i].min()
    max_val = X_wine[:, i].max()
    print(f"   • {feature_name}: {min_val:.1f} - {max_val:.1f}")

print()
print("💡 Problem: Riesige Unterschiede in den Skalen!")
print("   • Alkohol: ~11-15 Prozent")
print("   • Proline: ~278-1680 mg/L (100x größer!)")
print()
print("🔧 Lösung: StandardScaler vor PCA anwenden")

In [ ]:
# Daten skalieren (KRITISCH für PCA!)
scaler = StandardScaler()
X_wine_scaled = scaler.fit_transform(X_wine)

print("✅ Skalierung durchgeführt:")
print(f"   • Mittelwerte nach Skalierung: {X_wine_scaled.mean(axis=0)[:3]} ... (sollten ~0 sein)")
print(f"   • Std.-Abweichungen: {X_wine_scaled.std(axis=0)[:3]} ... (sollten ~1 sein)")
print("   • Alle Features sind jetzt gleichberechtigt!")

### PCA auf Wine-Datensatz: 13D → 2D

In [ ]:
# PCA anwenden: 13D → 2D
pca_wine = PCA(n_components=2)
X_wine_pca = pca_wine.fit_transform(X_wine_scaled)

print("🔍 PCA auf Wine-Datensatz Ergebnisse:")
print(f"   • Original Dimensionen: {X_wine_scaled.shape[1]}D")
print(f"   • Nach PCA: {X_wine_pca.shape[1]}D")
print()
print("📊 Explained Variance:")
for i, var in enumerate(pca_wine.explained_variance_ratio_):
    print(f"   • PC{i+1}: {var:.3f} ({var*100:.1f}%)")

total_variance = pca_wine.explained_variance_ratio_.sum()
print(f"   • PC1 + PC2 zusammen: {total_variance:.3f} ({total_variance*100:.1f}%)")
print()
print(f"💡 Mit nur 2D behalten wir {total_variance:.1%} der ursprünglichen Information!")
print("   Das ist sehr gut - wir verlieren nur wenig Information!")

In [ ]:
# Visualisierung: Wine-Datensatz in 2D nach PCA
plt.figure(figsize=(15, 5))

# Plot 1: PCA-Ergebnis ohne Farben (so sieht es PCA)
plt.subplot(1, 3, 1)
plt.scatter(X_wine_pca[:, 0], X_wine_pca[:, 1], alpha=0.7, s=50, c='gray')
plt.xlabel(f'PC1 ({pca_wine.explained_variance_ratio_[0]:.1%} Varianz)')
plt.ylabel(f'PC2 ({pca_wine.explained_variance_ratio_[1]:.1%} Varianz)')
plt.title('Wine-Daten in 2D (PCA)\n"Blindes" PCA - ohne Labels')
plt.grid(True, alpha=0.3)

# Plot 2: Mit echten Produzenten-Labels eingefärbt
plt.subplot(1, 3, 2)
colors = ['red', 'blue', 'green']
for i, (prod_name, color) in enumerate(zip(wine.target_names, colors)):
    mask = y_wine == i
    plt.scatter(X_wine_pca[mask, 0], X_wine_pca[mask, 1], 
               alpha=0.7, s=50, c=color, label=f'{prod_name}')
plt.xlabel(f'PC1 ({pca_wine.explained_variance_ratio_[0]:.1%} Varianz)')
plt.ylabel(f'PC2 ({pca_wine.explained_variance_ratio_[1]:.1%} Varianz)')
plt.title('Wine-Daten in 2D (PCA)\nMit echten Produzenten-Labels')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Explained Variance
plt.subplot(1, 3, 3)
components = range(1, len(pca_wine.explained_variance_ratio_) + 1)
plt.bar(components, pca_wine.explained_variance_ratio_, alpha=0.7)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance\nPro Komponente')
plt.xticks(components)
for i, var in enumerate(pca_wine.explained_variance_ratio_):
    plt.text(i+1, var + 0.01, f'{var:.1%}', ha='center', fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎯 Fantastisch! PCA hat die Weine automatisch in 2D gruppiert!")
print("   • Ohne die Produzenten-Labels zu kennen")
print("   • Die 3 Gruppen sind klar sichtbar")
print("   • PC1 + PC2 erklären zusammen über 50% der Varianz")

## 4. k-Means + PCA: Das Dream-Team!

Jetzt kombinieren wir **k-Means Clustering** mit **PCA Visualisierung**.

### Der Workflow:
1. 🔧 **Daten skalieren** (StandardScaler)
2. 🤖 **k-Means auf 13D-Daten** (alle Features nutzen)
3. 📊 **PCA für 2D-Visualisierung** (verstehen, was k-Means fand)
4. 📈 **Bewertung** (ARI + Silhouette Score)

In [ ]:
# k-Means auf den skalierten 13D-Daten anwenden
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
wine_clusters = kmeans.fit_predict(X_wine_scaled)

# Metriken berechnen
ari_score = adjusted_rand_score(y_wine, wine_clusters)
sil_score = silhouette_score(X_wine_scaled, wine_clusters)

print("🤖 k-Means Ergebnisse (13D):")
print(f"   • Adjusted Rand Index: {ari_score:.3f}")
print(f"   • Silhouette Score: {sil_score:.3f}")
print()

# Cluster-Verteilung
unique_clusters, cluster_counts = np.unique(wine_clusters, return_counts=True)
print("📊 k-Means Cluster-Größen:")
for cluster, count in zip(unique_clusters, cluster_counts):
    print(f"   • Cluster {cluster}: {count} Weine")

print()
print("🏭 Echte Produzenten-Verteilung (Referenz):")
for i, (name, count) in enumerate(zip(wine.target_names, counts)):
    print(f"   • {name}: {count} Weine")

In [ ]:
# Visualisierung: k-Means Cluster in PCA-2D-Raum
plt.figure(figsize=(15, 5))

# Plot 1: k-Means Cluster (so sieht k-Means die Gruppen)
plt.subplot(1, 3, 1)
colors_km = ['purple', 'orange', 'brown']
for cluster in range(3):
    mask = wine_clusters == cluster
    plt.scatter(X_wine_pca[mask, 0], X_wine_pca[mask, 1], 
               alpha=0.7, s=50, c=colors_km[cluster], 
               label=f'k-Means Cluster {cluster}')
plt.xlabel(f'PC1 ({pca_wine.explained_variance_ratio_[0]:.1%} Varianz)')
plt.ylabel(f'PC2 ({pca_wine.explained_variance_ratio_[1]:.1%} Varianz)')
plt.title('k-Means Cluster\n(in PCA 2D-Space)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Echte Produzenten (Ground Truth)
plt.subplot(1, 3, 2)
colors_true = ['red', 'blue', 'green']
for i, (prod_name, color) in enumerate(zip(wine.target_names, colors_true)):
    mask = y_wine == i
    plt.scatter(X_wine_pca[mask, 0], X_wine_pca[mask, 1], 
               alpha=0.7, s=50, c=color, label=f'{prod_name}')
plt.xlabel(f'PC1 ({pca_wine.explained_variance_ratio_[0]:.1%} Varianz)')
plt.ylabel(f'PC2 ({pca_wine.explained_variance_ratio_[1]:.1%} Varianz)')
plt.title('Echte Produzenten\n(Ground Truth)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Metriken-Vergleich
plt.subplot(1, 3, 3)
metrics = ['ARI', 'Silhouette']
values = [ari_score, sil_score]
colors_bar = ['lightcoral', 'lightblue']
bars = plt.bar(metrics, values, color=colors_bar, alpha=0.7)
plt.ylabel('Score')
plt.title('k-Means Performance\nMetriken')
plt.ylim(0, 1)

# Werte auf Balken schreiben
for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{value:.3f}', ha='center', fontsize=12, fontweight='bold')

# Interpretations-Linien
plt.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, label='Sehr gut (>0.8)')
plt.axhline(y=0.6, color='orange', linestyle='--', alpha=0.5, label='Gut (>0.6)')
plt.axhline(y=0.4, color='red', linestyle='--', alpha=0.5, label='Mäßig (>0.4)')
plt.legend(loc='upper left', fontsize=8)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎊 Ergebnis-Interpretation:")
print(f"   • ARI = {ari_score:.3f}: k-Means findet die echten Produzenten {'sehr gut' if ari_score > 0.8 else 'gut' if ari_score > 0.6 else 'mäßig'}!")
print(f"   • Silhouette = {sil_score:.3f}: Cluster sind {'sehr gut getrennt' if sil_score > 0.7 else 'gut getrennt' if sil_score > 0.5 else 'mäßig getrennt'}")
print("   • PCA zeigt: k-Means findet sinnvolle Gruppen in 2D!")

## 5. Explained Variance im Detail: Wie viele Komponenten brauchen wir?

Eine wichtige Frage bei PCA: **Wie viele Komponenten soll ich behalten?** Schauen wir uns die **Explained Variance** für alle 13 Komponenten an.

In [ ]:
# PCA mit allen Komponenten für Analyse
pca_full = PCA()
X_wine_pca_full = pca_full.fit_transform(X_wine_scaled)

# Kumulierte Explained Variance berechnen
explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print("📊 Explained Variance pro Komponente:")
for i, (var, cum_var) in enumerate(zip(explained_variance, cumulative_variance)):
    print(f"   PC{i+1:2d}: {var:.3f} ({var*100:5.1f}%) | Kumulativ: {cum_var:.3f} ({cum_var*100:5.1f}%)")

print()
print("🎯 Wichtige Schwellwerte:")
for threshold in [0.8, 0.9, 0.95]:
    n_components = np.argmax(cumulative_variance >= threshold) + 1
    print(f"   • Für {threshold:.0%} Varianz: {n_components} Komponenten nötig")

In [ ]:
# Visualisierung: Explained Variance Analysis
plt.figure(figsize=(15, 5))

# Plot 1: Explained Variance pro Komponente
plt.subplot(1, 2, 1)
components = range(1, len(explained_variance) + 1)
plt.bar(components, explained_variance, alpha=0.7, color='skyblue')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance\npro Komponente')
plt.xticks(range(1, 14))
plt.grid(True, alpha=0.3)

# Top-3 Komponenten markieren
for i in range(3):
    plt.text(i+1, explained_variance[i] + 0.005, f'{explained_variance[i]:.2f}', 
             ha='center', fontsize=9, fontweight='bold')

# Plot 2: Kumulierte Explained Variance
plt.subplot(1, 2, 2)
plt.plot(components, cumulative_variance, 'bo-', linewidth=2, markersize=6)
plt.xlabel('Anzahl Komponenten')
plt.ylabel('Kumulierte Explained Variance')
plt.title('Kumulierte Explained Variance\n(Wie viele Komponenten?)')

# Wichtige Schwellwerte markieren
thresholds = [0.8, 0.9, 0.95]
colors = ['red', 'orange', 'green']
for threshold, color in zip(thresholds, colors):
    plt.axhline(y=threshold, color=color, linestyle='--', alpha=0.7)
    n_comp = np.argmax(cumulative_variance >= threshold) + 1
    plt.axvline(x=n_comp, color=color, linestyle='--', alpha=0.7)
    plt.text(n_comp + 0.5, threshold, f'{threshold:.0%}\n({n_comp} PC)', 
             fontsize=9, color=color, fontweight='bold')

plt.grid(True, alpha=0.3)
plt.xticks(range(1, 14))

print("💡 PCA-Entscheidungshilfe:")
print("   • PC1 + PC2: 55% der Varianz → gut für Visualisierung")
print("   • PC1-PC4: 73% der Varianz → gut für weitere Analyse")
print("   • PC1-PC8: 90% der Varianz → gut für Machine Learning")
print("   • Alle 13 PC: 100% der Varianz → kein Informationsverlust")

## 6. Zusammenfassung und wichtige Erkenntnisse

### 🎓 Was haben wir gelernt?

**PCA-Grundprinzip:**
- ✅ **Findet die "wichtigsten Richtungen"** in hochdimensionalen Daten
- ✅ **Reduziert Dimensionen** ohne viel Information zu verlieren  
- ✅ **Macht Visualisierung möglich** (13D → 2D)

**Praktische Anwendung:**
- ✅ **Immer zuerst skalieren** (StandardScaler)
- ✅ **Explained Variance prüfen** (wie viel Info behalten wir?)
- ✅ **Mit k-Means kombinieren** (Clustering + Visualisierung)

**Wine-Datensatz Ergebnis:**
- ✅ **55% Varianz mit nur 2D** → sehr gut für Visualisierung
- ✅ **k-Means findet echte Produzenten** (ARI ≈ 0.87)
- ✅ **PCA macht Cluster sichtbar** in 2D-Raum

In [ ]:
print("🎊 Herzlichen Glückwunsch!")
print("Du hast erfolgreich PCA kennengelernt:")
print()
print("✅ Künstliche Daten: 2D → 1D (Konzept verstanden)")
print("✅ Wine-Datensatz: 13D → 2D (echte Anwendung)")
print("✅ Explained Variance: Wissen, wie viel Info verloren geht")
print("✅ k-Means + PCA: Das Dream-Team für unüberwachtes Lernen")
print()
print("🚀 Nächste Schritte:")
print("   • PCA auf eigene Datensätze anwenden")
print("   • Verschiedene n_components ausprobieren")
print("   • PCA + andere Clustering-Algorithmen kombinieren")
print("   • Feature-Interpretation in PC-Space lernen")
print()
print("💪 Du bist bereit für hochdimensionale Datenanalyse!")